## This ipynb help you to transfer the CONLL NER dataset to the backend's DB. So the trainer can train NER model base on the data.

In [6]:
from core.config import API_PORT

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./NER_multilabel_data_v2.csv")

In [3]:
df[90:100]

,Unnamed: 0,Sentence #,Word,POS,Tag,newTag
90,90,NaN,the,DT,O,O
91,91,NaN,annual,JJ,O,O
92,92,NaN,conference,NN,O,O
93,93,NaN,of,IN,O,O
94,94,NaN,Britain,NNP,B-geo,B-geo|Location|Party
95,95,NaN,'s,POS,O,O
96,96,NaN,ruling,VBG,O,O
97,97,NaN,Labor,NNP,B-org,B-org|Party
98,98,NaN,Party,NNP,I-org,I-org|Party
99,99,NaN,in,IN,O,O


In [4]:
df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

In [5]:
import numpy as np

In [10]:
sentences = list(set(df["Sentence #"]))

In [11]:
import requests

In [13]:
words = df[df["Sentence #"] == sentence]

In [14]:
def process_data(row):
    return {
        "text": row.Word,
        "labels": row.newTag.split("|")
    }

In [113]:
def post_data(sentence):
    data = {
        "user": "CONLL_0818@admin",
        "tags": ["setup"]
        "texts": list(df[df["Sentence #"] == sentence].apply(process_data, axis = 1))
    }
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    return requests.post(f"http://localhost:{API_PORT}/api/v1/data/labeledText", data=json.dumps(data), headers = headers)

In [115]:
# put datas into DB
import json
from tqdm import tqdm
import threading
tasks = []
for sentence in tqdm(sentences):
    t = threading.Thread(target=post_data, args={sentence})
    t.start()
    tasks.append(t)
for task in tqdm(tasks):
    task.join()

100%|███████████████████████████| 47959/47959 [00:00<00:00, 529359.85it/s]
